In [10]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("attention.pdf")
docs = loader.load()
# docs

In [7]:
# !pip install sentence-transformers==2.2.2
#InstructorEmbedding faiss-cpu

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)


In [12]:
documents=text_splitter.split_documents(docs)
# documents

In [13]:
import pickle
import faiss
from langchain.vectorstores import FAISS

In [ ]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
from InstructorEmbedding import INSTRUCTOR

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl", 
                                                      model_kwargs={"device": "cpu"})

In [4]:
def store_embeddings(docs, embeddings, sotre_name, path):
    
    vectorStore = FAISS.from_documents(docs, embeddings)

    with open(f"{path}/faiss_{sotre_name}.pkl", "wb") as f:
        pickle.dump(vectorStore, f)

        
def load_embeddings(sotre_name, path):
    with open(f"{path}/faiss_{sotre_name}.pkl", "rb") as f:
        VectorStore = pickle.load(f)
    return VectorStore

# store_embeddings(texts, 
#                  instructor_embeddings, 
#                  sotre_name='instructEmbeddings', 
#                  path=Embedding_store_path)

# db_instructEmbedd = load_embeddings(sotre_name='instructEmbeddings', 
#                                     path=Embedding_store_path)

In [ ]:
db_instructEmbedd = FAISS.from_documents(texts, instructor_embeddings)

In [5]:
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
llm=Ollama(model="llama2")
llm

Ollama()

In [6]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

In [ ]:
## Chain Introduction
## Create Stuff Docment Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [ ]:
retriever = db_instructEmbedd.as_retriever(search_kwargs={"k": 3})

In [ ]:
"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [ ]:
response=retrieval_chain.invoke({"input":"Scaled Dot-Product Attention"})

In [ ]:
response['answer']